In [35]:
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from nba_ou.prediction.clean_data_before_prediction import prepare_training_dataframe
import numpy as np

In [4]:
import pandas as pd
from nba_ou.create_training_data.create_df_to_predict import create_df_to_predict
# Create training data up to a specific date
date_to_train = "2026-05-10"

safe_limit = "2024-10-11"  # Optional: specify start date


df_total= create_df_to_predict(
    recent_limit_to_include=date_to_train, older_limit_to_include=safe_limit
)


Loaded 3470 odds records from database
Loading NBA data from PostgreSQL...
Loaded 7093 game records from database
Loaded 95786 player records from database
Successfully loaded 7093 games and 95786 player records
Loading data for seasons: ['2023-24', '2024-25', '2025-26']
Found 38 rows with potential home/away parsing errors.
Fixed HOME column for 38 problematic rows.
Overtime adjustments completed.


/home/adrian_alvarez/Projects/NBA_over_under_predictor/src/nba_ou/data_preparation/odds/process_odds_data.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odds.sort_values(by=["game_date", "team_home"], ascending=False, inplace=True)
/home/adrian_alvarez/Projects/NBA_over_under_predictor/src/nba_ou/data_preparation/odds/process_odds_data.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odds_processed["SPREAD_AWAY"] = -df_odds_processed["SPREAD_HOME"]
Computing rolling stats: 100%|██████████| 25/25 [00:17<00:00,  1.46it/s]
/home/adrian_alvarez/Projec

Loaded 28971 injury records from database


Adding players data: 100%|██████████| 5522/5522 [01:56<00:00, 47.46it/s] 
/home/adrian_alvarez/Projects/NBA_over_under_predictor/src/nba_ou/data_preparation/players/attach_player_features.py:224: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_team["TOTAL_INJURED_PLAYER_PTS_BEFORE"] = (
/home/adrian_alvarez/Projects/NBA_over_under_predictor/src/nba_ou/data_preparation/merged_home_away_data/merge_home_away.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["STAR_OFFENSIVE_RATIO_IMPROVEMENT_BEFORE"] = (
/home/adrian_alvarez/

Loaded 10771 referee records from database


Computing referee features: 100%|██████████| 2761/2761 [00:56<00:00, 48.63it/s]



Referee features successfully merged into training data!
Training data shape after merge: (2761, 994)


Computing absence effects: 100%|██████████| 2761/2761 [00:35<00:00, 77.46it/s] 


Computing travel features...
Travel features computed successfully.

----------------------------------------
Training data created up to 2026-05-10 00:00:00
Number of games: 2761
Number of features: 973
----------------------------------------



In [63]:
older_limit_to_include = "2025-12-10"  # Optional: specify start date
max_limit_date = "2026-01-24"

df_total['GAME_DATE'] = pd.to_datetime(df_total['GAME_DATE'])
df_eval = df_total[(df_total['GAME_DATE'] >= pd.to_datetime(older_limit_to_include)) & (df_total['GAME_DATE'] <= pd.to_datetime(max_limit_date))]
df_eval = prepare_training_dataframe(df_eval)

/home/adrian_alvarez/Projects/NBA_over_under_predictor/src/nba_ou/prediction/clean_data_before_prediction.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stats.dropna(subset=[total_line_col], inplace=True)


In [64]:
#Reosrder columns to have total points the first one
df_eval = df_eval[['TOTAL_POINTS'] + [col for col in df_eval.columns if col != 'TOTAL_POINTS']]
df_eval

,TOTAL_POINTS,TOTAL_OVER_UNDER_LINE,GAME_NUMBER_TEAM_HOME,GAME_NUMBER_TEAM_AWAY,IS_PLAYOFF_GAME,PLAYOFF_GAMES_LAST_SEASON_TEAM_AWAY,PLAYOFF_GAMES_LAST_SEASON_TEAM_HOME,SEASON_YEAR,WINS_BEFORE_THIS_GAME_TEAM_HOME,LOSSES_BEFORE_THIS_GAME_TEAM_HOME,...,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH,IS_US_HOLIDAY
61,215,212.9999,43,41,0,7.0,0.0,2025,25,17,...,0.000000,0,0.317697,0.424849,-0.041652,55.684883,56.300063,0,1,0
8,202,213.4999,44,46,0,14.0,5.0,2025,18,25,...,0.857313,1,0.313847,0.460587,0.097994,60.586948,61.878204,0,1,0
38,243,213.9999,41,43,0,0.0,0.0,2025,12,28,...,0.720235,-1,0.141408,0.293617,0.012993,54.191875,55.343818,0,1,1
109,190,214.4999,38,38,0,7.0,6.0,2025,28,9,...,0.234741,2,0.495857,0.234389,-0.156275,58.487861,56.322250,1,1,0
113,226,214.4999,35,37,0,7.0,0.0,2025,11,23,...,0.941327,0,0.083092,0.235720,-0.014296,54.521985,55.717979,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,259,249.4999,31,31,0,0.0,9.0,2025,16,14,...,0.000000,0,0.225466,0.313400,0.066354,58.434898,54.724389,0,12,0
94,235,249.4999,41,39,0,0.0,9.0,2025,22,18,...,0.000000,0,0.097326,0.171491,0.000490,59.718539,56.807117,0,1,0
226,237,250.4999,32,31,0,4.0,0.0,2025,15,16,...,0.000000,0,0.029358,0.020950,0.004899,59.594000,59.963627,0,12,0
251,247,251.4999,28,28,0,0.0,14.0,2025,20,7,...,0.000000,0,0.285059,0.231051,-0.012917,60.818235,56.259298,0,12,0


In [65]:
df_eval['OVER_UNDER'] = np.where(df_eval['TOTAL_POINTS'] >= df_eval['TOTAL_OVER_UNDER_LINE'], 1, 0) # 1 if over, 0 if under
df_eval[['TOTAL_OVER_UNDER_LINE', 'TOTAL_POINTS','OVER_UNDER']]

,TOTAL_OVER_UNDER_LINE,TOTAL_POINTS,OVER_UNDER
61,212.9999,215,1
8,213.4999,202,0
38,213.9999,243,1
109,214.4999,190,0
113,214.4999,226,1
...,...,...,...
246,249.4999,259,1
94,249.4999,235,0
226,250.4999,237,0
251,251.4999,247,0


# Predicitons Classifier

In [66]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor.load("/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/classifier_autoglue")

In [67]:
df_predict = df_eval.drop(columns=['OVER_UNDER', 'TOTAL_POINTS'])
y_test = df_eval['OVER_UNDER']

In [68]:
y_test

61     1
8      0
38     1
109    0
113    1
      ..
246    1
94     0
226    0
251    0
234    0
Name: OVER_UNDER, Length: 315, dtype: int64

In [69]:
y_test_pred = predictor.predict(df_predict)
y_test_pred

61     1
8      1
38     1
109    1
113    1
      ..
246    1
94     0
226    0
251    1
234    1
Name: target, Length: 315, dtype: int64

In [70]:
metrics = {
    "accuracy": accuracy_score(y_test, y_test_pred),
    "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1": f1_score(y_test, y_test_pred),
    "error_rate": 1 - accuracy_score(y_test, y_test_pred),
}

pd.Series(metrics).round(4)

accuracy             0.6349
balanced_accuracy    0.6214
precision            0.5868
recall               0.5221
f1                   0.5525
error_rate           0.3651
dtype: float64

## Make Predictions and Evaluate

In [93]:
import numpy as np
def sign_flip_weighted_mse_obj_sklearn(y_true, y_pred, w_same=0.5, w_flip=3.0, eps_push=0.5):
    """
    Custom objective for XGBRegressor: signature (y_true, y_pred) -> (grad, hess)
    Loss per sample: w * (y_pred - y_true)^2

      - If |y_true| < eps_push: neutral, w = 1.0
      - Else if y_true and y_pred have same sign: w = w_same
      - Else (sign flip): w = w_flip
    """
    y_true = np.asarray(y_true, dtype=np.float32)
    y_pred = np.asarray(y_pred, dtype=np.float32)

    err = y_pred - y_true

    neutral = np.abs(y_true) < eps_push
    wrong = (~neutral) & ((y_true * y_pred) < 0)

    w = np.ones_like(y_true, dtype=np.float32)
    w[~neutral & ~wrong] = np.float32(w_same)
    w[wrong] = np.float32(w_flip)

    grad = 2.0 * w * err
    hess = 2.0 * w
    return grad, hess

## Load Models and Prepare Data

In [94]:
import pickle
import numpy as np
import pandas as pd

# Load production models
# model_path_full = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/total_points_bet_accuracy/best_xgboost_regressor_total_bet_2026-01-21_production.pkl"
# model_path_important = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/total_points_bet_accuracy/best_xgboost_regressor_total_bet_important_2026-01-21_production.pkl"


model_path_total =  "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/total_points_bet_accuracy/best_xgboost_regressor_total_bet_2026-01-21_production_till_2025.pkl"
model_path_dif =  "/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/dif_to_line/best_xgboost_regressor_dif_2026-01-22_production_till_2025.pkl"


with open(model_path_total, "rb") as f:
    model_full = pickle.load(f)
    
with open(model_path_dif, "rb") as f:
    model_dif = pickle.load(f)

In [95]:
# Prepare evaluation data (similar to training process)
from nba_ou.data_preparation.missing_data.handle_missing_data import apply_missing_policy, compute_and_save_train_medians

df_eval_prepared = df_eval.copy()

# Filter and clean data
df_eval_prepared = df_eval_prepared[df_eval_prepared['TOTAL_POINTS'] > 130]
df_eval_prepared.dropna(subset=['TOTAL_OVER_UNDER_LINE'], inplace=True)
df_eval_prepared = df_eval_prepared[df_eval_prepared['TOTAL_OVER_UNDER_LINE'] > 100]

print(f"Evaluation data shape: {df_eval_prepared.shape}")

Evaluation data shape: (45, 974)


In [96]:
df_eval_prepared.fillna(0)

,TOTAL_OVER_UNDER_LINE,TEAM_ID_TEAM_HOME,TEAM_CITY_TEAM_HOME,TEAM_ABBREVIATION_TEAM_HOME,TEAM_NAME_TEAM_HOME,MATCHUP_TEAM_HOME,GAME_NUMBER_TEAM_HOME,TEAM_ID_TEAM_AWAY,TEAM_CITY_TEAM_AWAY,TEAM_ABBREVIATION_TEAM_AWAY,...,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH,IS_US_HOLIDAY,IS_OVER
61,212.9999,1610612761,Toronto,TOR,Toronto Raptors,TOR vs. LAC,43,1610612746,LA,LAC,...,0,0.317697,0.424849,-0.041652,55.684883,56.300063,0,1,0,True
67,218.9999,1610612765,Detroit,DET,Detroit Pistons,DET vs. PHX,39,1610612756,Phoenix,PHX,...,3,0.111882,0.551390,0.040899,57.978106,55.556400,0,1,0,False
104,218.9999,1610612743,Denver,DEN,Denver Nuggets,DEN vs. MIL,39,1610612749,Milwaukee,MIL,...,0,0.549681,0.084513,-0.118070,60.718366,57.176250,1,1,0,False
90,219.4999,1610612742,Dallas,DAL,Dallas Mavericks,DAL vs. BKN,40,1610612751,Brooklyn,BKN,...,1,0.424130,0.360083,-0.067197,57.251023,50.222031,0,1,0,False
103,219.9999,1610612761,Toronto,TOR,Toronto Raptors,TOR vs. PHI,40,1610612755,Philadelphia,PHI,...,0,0.454727,0.218133,-0.097958,56.810799,56.454541,1,1,0,False
75,220.4999,1610612754,Indiana,IND,Indiana Pacers,IND vs. TOR,41,1610612761,Toronto,TOR,...,0,0.380217,0.355427,0.017689,57.089230,56.760584,0,1,0,False
91,221.4999,1610612761,Toronto,TOR,Toronto Raptors,TOR vs. PHI,41,1610612755,Philadelphia,PHI,...,0,0.316198,0.014465,-0.076598,56.138316,56.048065,0,1,0,False
60,221.4999,1610612745,Houston,HOU,Houston Rockets,HOU vs. MIN,39,1610612750,Minnesota,MIN,...,-2,0.104761,0.274772,0.041572,56.436948,62.226701,0,1,0,False
102,221.4999,1610612763,Memphis,MEM,Memphis Grizzlies,MEM vs. BKN,39,1610612751,Brooklyn,BKN,...,0,0.323021,0.150116,-0.018147,58.025924,57.141511,1,1,0,False
92,221.4999,1610612746,LA,LAC,Los Angeles Clippers,LAC vs. CHA,39,1610612766,Charlotte,CHA,...,0,0.226503,0.220694,0.073684,59.324148,58.156866,0,1,0,True


In [97]:
# Drop unnecessary columns
text_columns = df_eval_prepared.select_dtypes(include=['object', 'string']).columns

cols_to_drop = ['SEASON_ID', 'GAME_ID', 'SEASON_TYPE', 'TEAM_ID_TEAM_HOME',
                'TEAM_ID_TEAM_AWAY', 'IS_OVERTIME']

cols_to_drop += [
    "MATCHUP_TEAM_HOME", "TEAM_CITY_TEAM_AWAY",
    "TEAM_ABBREVIATION_TEAM_AWAY", "TEAM_ABBREVIATION_TEAM_HOME",
    "TEAM_CITY_TEAM_HOME", "TEAM_NAME_TEAM_AWAY", "MATCHUP_TEAM_AWAY",
    'TEAM_NAME_TEAM_HOME'
]
cols_to_drop += list(text_columns)


# Keep GAME_DATE for now, we'll drop it after
game_dates = df_eval_prepared['GAME_DATE'].copy()
df_eval_prepared.drop(columns=[col for col in cols_to_drop if col in df_eval_prepared.columns], inplace=True, errors='ignore')

In [98]:
# Handle missing data
train_medians = compute_and_save_train_medians(df_eval_prepared)

df_eval_prepared, report = apply_missing_policy(
    df_eval_prepared,
    train_medians=train_medians,
    current_total_line_col="TOTAL_OVER_UNDER_LINE",
    drop_mode="strict",
)

print(f"Data shape after handling missing data: {df_eval_prepared.shape}")

Data shape after handling missing data: (45, 625)


In [99]:
# Drop GAME_DATE and prepare X, y
df_eval_prepared.drop(columns='GAME_DATE', inplace=True, errors='ignore')

X_eval = df_eval_prepared.drop(['TOTAL_POINTS'], axis=1, errors='ignore')
y_eval = df_eval_prepared['TOTAL_POINTS']

print(f"X_eval shape: {X_eval.shape}")
print(f"y_eval shape: {y_eval.shape}")

X_eval shape: (45, 623)
y_eval shape: (45,)


In [100]:
df_eval_prepared['LINE_ERROR'] = df_eval_prepared['TOTAL_POINTS'] - df_eval_prepared['TOTAL_OVER_UNDER_LINE']


X_eval_dif = df_eval_prepared.drop([['TOTAL_POINTS', 'LINE_ERROR'] ], axis=1, errors='ignore')
y_eval_dif = df_eval_prepared['LINE_ERROR']

In [101]:
# Over/Under betting accuracy function
def over_under_betting_accuracy(y_true, y_pred, betting_line):
    """
    Calculate the accuracy of over/under betting decisions.
    
    A bet is successful if:
    - We predict OVER (pred > line) and actual is OVER (true > line), OR
    - We predict UNDER (pred < line) and actual is UNDER (true < line)
    """
    pred_diff = y_pred - betting_line
    true_diff = y_true - betting_line
    
    correct_predictions = np.sign(pred_diff) == np.sign(true_diff)
    accuracy = np.mean(correct_predictions)
    
    return accuracy

In [102]:
# Model 1: Full features
y_pred_full = model_full.predict(X_eval)
betting_line = X_eval['TOTAL_OVER_UNDER_LINE'].values

# Calculate metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

mae_full = mean_absolute_error(y_eval, y_pred_full)
mse_full = mean_squared_error(y_eval, y_pred_full)
rmse_full = root_mean_squared_error(y_eval, y_pred_full)
ou_accuracy_full = over_under_betting_accuracy(y_eval, y_pred_full, betting_line)

print("=" * 50)
print("MODEL 1: Full Features")
print("=" * 50)
print(f"MAE: {mae_full:.4f}")
print(f"MSE: {mse_full:.4f}")
print(f"RMSE: {rmse_full:.4f}")
print(f"Over/Under Betting Accuracy: {ou_accuracy_full:.2%}")
print()

ValueError: feature_names mismatch: ['TOTAL_OVER_UNDER_LINE', 'GAME_NUMBER_TEAM_AWAY', 'IS_PLAYOFF_GAME', 'PLAYOFF_GAMES_LAST_SEASON_TEAM_AWAY', 'PLAYOFF_GAMES_LAST_SEASON_TEAM_HOME', 'SEASON_YEAR', 'WINS_BEFORE_THIS_GAME_TEAM_HOME', 'LOSSES_BEFORE_THIS_GAME_TEAM_HOME', 'TEAM_RECORD_BEFORE_GAME_TEAM_HOME', 'REST_DAYS_BEFORE_MATCH_TEAM_HOME', 'PTS_LAST_ALL_5_MATCHES_BEFORE_TEAM_HOME', 'PTS_LAST_HOME_AWAY_5_MATCHES_BEFORE_TEAM_HOME', 'PTS_SEASON_BEFORE_AVG_TEAM_HOME', 'PTS_LAST_ALL_10_MATCHES_BEFORE_TEAM_HOME', 'PTS_LAST_HOME_AWAY_10_MATCHES_BEFORE_TEAM_HOME', 'PTS_LAST_10_WMA_BEFORE_TEAM_HOME', 'PTS_LAST_HOME_AWAY_10_WMA_BEFORE_TEAM_HOME', 'PTS_LAST_5_WMA_BEFORE_TEAM_HOME', 'PTS_LAST_HOME_AWAY_5_WMA_BEFORE_TEAM_HOME', 'OFF_RATING_LAST_ALL_5_MATCHES_BEFORE_TEAM_HOME', 'OFF_RATING_LAST_HOME_AWAY_5_MATCHES_BEFORE_TEAM_HOME', 'OFF_RATING_SEASON_BEFORE_AVG_TEAM_HOME', 'OFF_RATING_LAST_ALL_10_MATCHES_BEFORE_TEAM_HOME', 'OFF_RATING_LAST_HOME_AWAY_10_MATCHES_BEFORE_TEAM_HOME', 'DEF_RATING_LAST_

In [18]:
# Model 2: Important features only
trained_features_important = model_dif.feature_names_in_
X_eval_dif = X_eval_dif[trained_features_important]

y_pred_diff = model_dif.predict(X_eval_dif)

y_pred_total_dif = y_pred_diff + X_eval_dif['TOTAL_OVER_UNDER_LINE']

mae_important = mean_absolute_error(y_eval_dif, y_pred_total_dif)
mse_important = mean_squared_error(y_eval_dif, y_pred_total_dif)
rmse_important = root_mean_squared_error(y_eval_dif, y_pred_total_dif)
# ou_accuracy_important = over_under_betting_accuracy(y_eval, y_pred_important, betting_line)
ou_accuracy_important = over_under_betting_accuracy(y_eval_dif, y_pred_total_dif, betting_line)



print("=" * 50)
print("MODEL 2: Important Features Only")
print("=" * 50)
print(f"MAE: {mae_important:.4f}")
print(f"MSE: {mse_important:.4f}")
print(f"RMSE: {rmse_important:.4f}")
print(f"Over/Under Betting Accuracy: {ou_accuracy_important:.2%}")
print()

MODEL 2: Important Features Only
MAE: 233.0131
MSE: 54541.9466
RMSE: 233.5422
Over/Under Betting Accuracy: 61.24%



In [166]:
# Create comparison dataframe
results_df = pd.DataFrame({
    'Game': range(len(y_eval)),
    'Actual_Total': y_eval.values,
    'Betting_Line': betting_line,
    'Pred_Full': y_pred_full,
    'Pred_Important': y_pred_important,
    'Actual_Over': (y_eval.values > betting_line).astype(int),
    'Pred_Over_Full': (y_pred_full > betting_line).astype(int),
    'Pred_Over_Important': (y_pred_important > betting_line).astype(int),
})

results_df['Correct_Full'] = (results_df['Actual_Over'] == results_df['Pred_Over_Full']).astype(int)
results_df['Correct_Important'] = (results_df['Actual_Over'] == results_df['Pred_Over_Important']).astype(int)

results_df

,Game,Actual_Total,Betting_Line,Pred_Full,Pred_Important,Actual_Over,Pred_Over_Full,Pred_Over_Important,Correct_Full,Correct_Important
0,0,249,238.9999,246.465897,248.463257,1,1,1,1,1
1,1,222,228.4999,223.058838,222.651459,0,0,0,1,1
2,2,248,224.4999,223.466217,223.289337,1,0,0,0,0
3,3,217,220.9999,222.631790,222.141708,0,1,1,0,0
4,4,226,222.9999,224.236160,224.233109,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
341,341,258,234.4999,237.924927,239.030182,1,1,1,1,1
342,342,219,228.4999,223.193588,223.231735,0,0,0,1,1
343,343,254,231.4999,231.988785,234.198898,1,1,1,1,1
344,344,255,232.4999,229.288177,231.851929,1,0,0,0,0


In [167]:
# Summary statistics
print("=" * 50)
print("VALIDATION SUMMARY")
print("=" * 50)
print(f"Total games evaluated: {len(y_eval)}")
print(f"\nActual games that went OVER: {(y_eval.values > betting_line).sum()}")
print(f"Actual games that went UNDER: {(y_eval.values <= betting_line).sum()}")
print(f"\nModel Full - Correct predictions: {results_df['Correct_Full'].sum()}/{len(y_eval)}")
print(f"Model Important - Correct predictions: {results_df['Correct_Important'].sum()}/{len(y_eval)}")

VALIDATION SUMMARY
Total games evaluated: 346

Actual games that went OVER: 153
Actual games that went UNDER: 193

Model Full - Correct predictions: 227/346
Model Important - Correct predictions: 256/346


In [168]:
results_df.to_csv("evaluation_results_recent_games.csv", index=False)